In [1]:
import pandas as pd
data2 = pd.read_csv('stock_prices_cash_180330.csv')
data2.head()

D:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,id,inL,inLPct,inM,inMPct,inS,inSPct,inSubOutL,inSubOutM,...,chg,close,high,low,open,percent,stockcode,timestamp,turnoverrate,volume
0,20180330,NaN,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,...,5.75,18.81,18.81,15.67,15.67,44.03,SZ300504,2018-03-29 16:00:00,0.05,35309
1,20180330,NaN,0.0,0.00,0.0,0.00,2394.0,0.43,0.0,0.0,...,8.78,28.73,28.73,23.94,23.94,44.01,SH603214,2018-03-29 16:00:00,0.08,19504
2,20180330,NaN,78782322.0,27.88,69522088.0,24.61,12026740.0,4.26,33560914.0,3214133.0,...,0.56,6.11,6.11,5.56,5.57,10.09,SZ300263,2018-03-29 16:00:00,9.20,47392412
3,20180329,NaN,2746297.0,4.22,27064211.0,41.55,7609108.0,11.68,2746297.0,5388110.0,...,0.13,5.55,5.58,5.44,5.44,2.40,SZ300263,2018-03-28 16:00:00,2.27,11753354
4,20180328,NaN,1772910.0,3.35,20161583.0,38.12,6751494.0,12.77,1772910.0,842825.0,...,0.02,5.42,5.50,5.31,5.36,0.37,SZ300263,2018-03-27 16:00:00,1.88,9762221


In [5]:
index[0]

{'date': 20170801,
 'down_stocks_gt_3': 0,
 'stocks': 3,
 'up_stocks_gt_0': 1,
 'up_stocks_gt_3': 0}

In [29]:
import json
index = []
for i in set(data2['date']):  
    if i > 20180209:
        date_index = {'date':str(i)}
        q = data2[data2['date']==i]
        date_index['stocks'] = len(q)
        date_index['up_stocks_gt_0'] = len(q[q['percent']>0])
        date_index['up_stocks_gt_3'] = len(q[q['percent']>3])
        date_index['down_stocks_gt_3'] = len(q[q['percent']<-3])
        index.append(date_index)

indexs = pd.read_json(json.dumps(index))


import datetime
def convert_time(x):
    return datetime.date(int(x[:4]), int(x[4:6]),int(x[-2:]))
indexs['date_index'] = indexs['date'].apply(lambda x:convert_time(str(x)))

indexs = indexs.set_index('date_index')
indexs['up_ratio'] = indexs['up_stocks_gt_0']*1.0/indexs['stocks']
indexs['up_3_ratio'] = indexs['up_stocks_gt_3']*1.0/indexs['stocks']
indexs['up_down_ratio'] = (indexs['up_stocks_gt_3'].fillna(0)+1)*1.0/(indexs['down_stocks_gt_3'].fillna(0)+1)
indexs = indexs.sort_index()
indexs.head()

,date,down_stocks_gt_3,stocks,up_stocks_gt_0,up_stocks_gt_3,up_ratio,up_3_ratio,up_down_ratio
date_index,,,,,,,,
2018-02-12,20180212,27,3164,2970,1124,0.938685,0.355247,40.178571
2018-02-13,20180213,141,3171,1542,204,0.486282,0.064333,1.443662
2018-02-14,20180214,131,3185,1782,175,0.559498,0.054945,1.333333
2018-02-22,20180222,26,3196,2987,580,0.934606,0.181477,21.518519
2018-02-23,20180223,42,3205,1835,238,0.572543,0.074259,5.558140


In [31]:
import tushare as ts
sh = ts.get_hist_data('sh',start='2018-02-12')
sh = sh.sort_index()
sh.head()

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
date,,,,,,,,,,,,,
2018-02-12,3128.37,3168.13,3154.13,3113.61,1533108.25,24.28,0.78,3245.188,3359.134,3433.599,2304915.40,2232380.08,2291881.74
2018-02-13,3176.11,3219.22,3184.96,3176.11,1515276.00,30.83,0.98,3208.050,3328.829,3421.018,2046859.65,2197537.70,2261907.80
2018-02-14,3188.25,3203.50,3199.16,3171.38,1003228.25,14.20,0.45,3186.030,3300.662,3408.742,1725625.85,2090607.13,2181546.70
2018-02-22,3237.57,3269.92,3268.56,3234.12,1387304.50,69.40,2.17,3187.332,3282.820,3398.433,1600561.75,1968832.88,2140892.15
2018-02-23,3275.43,3294.13,3289.02,3258.49,1457903.62,20.46,0.63,3219.166,3265.514,3388.491,1379364.12,1906502.69,2090018.97


In [32]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column,row
output_notebook()
import numpy as np

p1 = figure(width=1000, height=150, x_axis_type="datetime")
p1.line(indexs.index, indexs['up_ratio'],line_width=2)
p2 = figure(width=1000, height=150, x_axis_type="datetime")
p2.line(indexs.index, np.log(indexs['up_down_ratio']),color='green',line_width=2)
p3 = figure(width=1000, height=150, x_axis_type="datetime")
p3.line(pd.to_datetime(sh.index), sh['close'],color='red',line_width=2)
show(column(p1,p2,p3))

Loading BokehJS ...

In [33]:
import tushare as ts
today = ts.get_today_all()
today.head()

[Getting data:]############################################################

,code,name,changepercent,trade,open,high,low,settlement,volume,turnoverratio,amount,per,pb,mktcap,nmc
0,603999,读者传媒,1.240,7.35,7.29,7.39,7.29,7.26,2416280,1.04873,17755031,25.345,2.538,4.233600e+05,1.693440e+05
1,603998,方盛制药,0.000,9.43,0.00,0.00,0.00,9.43,0,0.00000,0,58.938,4.066,4.063731e+05,4.009605e+05
2,603997,继峰股份,-0.549,12.69,12.76,12.93,12.55,12.76,7371339,1.17005,93912659,21.508,4.848,8.092657e+05,7.994700e+05
3,603996,中新科技,0.118,16.92,16.88,17.16,16.82,16.90,3614000,3.49939,61379047,43.385,3.576,5.078538e+05,1.747413e+05
4,603993,洛阳钼业,-0.235,8.48,8.55,8.64,8.41,8.50,158269269,1.22180,1349048064,60.571,4.913,1.831616e+07,1.098476e+07


In [40]:
stocks = today[(today['per']>0) & (today['per']<1000)].sort_values(['per'],ascending=False)

In [43]:
#不同市盈率股票价格变动
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

btc_trace = go.Scatter(x=np.log(stocks.per), y=np.log(stocks.pb),mode='markers')
py.iplot([btc_trace])
